In [1]:
import torch.nn as nn
from tqdm import tqdm
from e3nn.nn import _fc
import torch
from torch_geometric.data.data import Data
from models.geometricv2 import GeometricNetV2
from dataset import Dataset, create_transform
from train import load_indexes
from torch_geometric.loader import DataLoader
from IPython.display import clear_output

DATA_DIR = 'data/ala_dipep'
TEST_ON = 'all'
N_SAMPLES = 2000
BATCH_SIZE = 10
LABELS_FILE = 'phi-psi-free-energy.txt'
BONDS_FILE = 'ala_dipep_bonds.csv'
PARTIAL_CHARGES_FILE = 'ala_dipep_partial_charges.csv'
ENERGY_LEVELS = 40
LR = 1e-3
CHECKPOINT = None #'tmpdir-all-1124-1745/checkpoint/epoch=199-step=41799.ckpt'
WEIGHTS = None

In [2]:
train_indexes, validation_indexes, test_indexes = load_indexes(DATA_DIR, TEST_ON, n_samples=N_SAMPLES)
transform = create_transform(DATA_DIR, LABELS_FILE, BONDS_FILE, PARTIAL_CHARGES_FILE, use_dihedrals=False, energy_levels=ENERGY_LEVELS)

test_dataset = Dataset(data_dir=DATA_DIR, indexes=test_indexes, transform=transform, use_dihedrals=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

loading indexes
indexes loaded


In [3]:
if CHECKPOINT:
        assert not WEIGHTS, 'Params --weights and --checkpoint are mutually exclusive'
        print(f'Loading model from checkpoint: {CHECKPOINT}')
        model = GeometricNetV2.load_from_checkpoint(strict=False, checkpoint_path=CHECKPOINT, sample=next(iter(test_loader)), lr=LR)
else:
    model = GeometricNetV2(next(iter(test_loader)), LR)
    if WEIGHTS:
        print(f'Loading model weights  {WEIGHTS}')
        try:
            model.load_state_dict(torch.load(WEIGHTS), strict=True)
            print(f'Model weights {WEIGHTS} loaded')
        except Exception as e:
            print(f'Model weights could not be loaded: {str(e)}')
    else:
        print('Initializing new model')

Initializing new model


In [4]:
def show_weights(model):
    for m in reversed(list(model.modules())):
        if isinstance(m, nn.Linear):
            print('Linear', torch.max(m.weight.grad[0][:10]))
        elif isinstance(m, _fc._Layer):
            print('GraphLinear', torch.max(m.weight.grad))
            break

In [5]:
optimizers, _ = model.configure_optimizers()
optimizer = optimizers[0]

model = model.cuda()
model.train()
for idx, data in tqdm(enumerate(test_loader)):
    if isinstance(data, list):
        for d in data:
            d = d.cuda()
    elif isinstance(data, Data):
        data = data.cuda()
    # run inference and sum up batch loss
    out = model.training_step(data, idx)
    loss = out['loss']

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if idx % 25 == 0:
        clear_output(wait=False)
        print(loss.item())
        show_weights(model)

126it [02:45,  1.10s/it]

8.006976127624512
Linear tensor(0.0073, device='cuda:0')
Linear tensor(0.0049, device='cuda:0')
Linear tensor(8.3500e-06, device='cuda:0')
GraphLinear tensor(9.8832e-05, device='cuda:0')
tensor([0.0428, 0.1266, 0.2303, 0.1650, 0.1317, 0.0642, 0.0488, 0.0424, 0.0275,
        0.0129, 0.0074, 0.0047, 0.0040, 0.0041, 0.0041, 0.0036, 0.0036, 0.0036,
        0.0032, 0.0035, 0.0033, 0.0033, 0.0033, 0.0033, 0.0035, 0.0033, 0.0032,
        0.0032, 0.0033, 0.0032, 0.0034, 0.0034, 0.0031, 0.0034, 0.0035, 0.0034,
        0.0031, 0.0033, 0.0034, 0.0033], device='cuda:0',
       grad_fn=<SelectBackward>) tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0340, 0.9300, 0.0360, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0')


127it [02:46,  1.10s/it]

tensor([0.0425, 0.1211, 0.1958, 0.1688, 0.1262, 0.0644, 0.0532, 0.0475, 0.0308,
        0.0144, 0.0087, 0.0059, 0.0051, 0.0050, 0.0052, 0.0045, 0.0045, 0.0044,
        0.0040, 0.0043, 0.0042, 0.0042, 0.0042, 0.0041, 0.0044, 0.0041, 0.0041,
        0.0041, 0.0042, 0.0040, 0.0043, 0.0042, 0.0040, 0.0042, 0.0044, 0.0043,
        0.0039, 0.0042, 0.0042, 0.0042], device='cuda:0',
       grad_fn=<SelectBackward>) tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.3420, 0.6560, 0.0020, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0')


128it [02:48,  1.10s/it]

...

142it [03:03,  1.10s/it]

tensor([0.0559, 0.1727, 0.1714, 0.1512, 0.1030, 0.0667, 0.0538, 0.0488, 0.0274,
        0.0129, 0.0082, 0.0062, 0.0058, 0.0054, 0.0053, 0.0046, 0.0042, 0.0044,
        0.0041, 0.0042, 0.0042, 0.0042, 0.0041, 0.0040, 0.0043, 0.0043, 0.0042,
        0.0041, 0.0042, 0.0041, 0.0042, 0.0042, 0.0041, 0.0042, 0.0043, 0.0043,
        0.0041, 0.0041, 0.0042, 0.0042], device='cuda:0',
       grad_fn=<SelectBackward>) tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0470, 0.9270, 0.0250, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0')


143it [03:04,  1.10s/it]

tensor([0.0551, 0.1836, 0.1857, 0.1501, 0.1031, 0.0656, 0.0512, 0.0458, 0.0256,
        0.0119, 0.0077, 0.0060, 0.0056, 0.0049, 0.0046, 0.0040, 0.0039, 0.0041,
        0.0037, 0.0038, 0.0037, 0.0038, 0.0036, 0.0036, 0.0037, 0.0038, 0.0036,
        0.0036, 0.0037, 0.0037, 0.0037, 0.0038, 0.0036, 0.0037, 0.0038, 0.0037,
        0.0037, 0.0036, 0.0038, 0.0037], device='cuda:0',
       grad_fn=<SelectBackward>) tensor([0.0000, 0.0000, 0.0000, 0.3800, 0.6190, 0.0010, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0')


144it [03:05,  1.29s/it]
